In [1]:
.libPaths('/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library')

In [2]:
library(IRkernel)
library(jsonlite)
library(repr)

In [3]:
##########联动isoform信息与RES
#deEx_iso:经过if usage差异表达得到的587个iso，进一步对dif+p值过滤，得到显著差异表达的125个iso
#其中71个iso具有IR
deEx_iso_IR <- read.csv("/disk1/wenqing/tmp_data/ASD/deEx_isoform_IR.csv")
dim(deEx_iso_IR)


[1] 71 36

In [10]:
length(unique(deEx_iso_IR$isoform_id))

[1] 71

In [4]:
deEx_iso <- read.csv("/disk1/wenqing/tmp_data/ASD/deEx_isoform.csv")
dim(deEx_iso)

[1] 125  31

In [5]:
#将差异表达的iso分为发生IR和未检测到IR的两组
iso_IR <- deEx_iso$isoform_id[deEx_iso$isoform_id %in% deEx_iso_IR$isoform_id]
iso_no_IR <- deEx_iso$isoform_id[!(deEx_iso$isoform_id %in% deEx_iso_IR$isoform_id)]

In [6]:
length(iso_IR)
length(iso_no_IR)

[1] 12

[1] 113

In [7]:
#调取两组iso的exon边界，构建剪接供受体位点
exon_region <- read.csv("/disk1/wenqing/tmp_data/ASD/switchList_exon.csv")
head(exon_region)

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,chr1,12010,12057,48,+,ENST00000450305.2_2,DDX11L1,DDX11L1
2,2,chr1,12179,12227,49,+,ENST00000450305.2_2,DDX11L1,DDX11L1
3,3,chr1,12613,12697,85,+,ENST00000450305.2_2,DDX11L1,DDX11L1
4,4,chr1,12975,13052,78,+,ENST00000450305.2_2,DDX11L1,DDX11L1
5,5,chr1,13221,13374,154,+,ENST00000450305.2_2,DDX11L1,DDX11L1
6,6,chr1,13453,13670,218,+,ENST00000450305.2_2,DDX11L1,DDX11L1


In [8]:
iso_IR_exon_region <- exon_region[exon_region$isoform_id %in% iso_IR,]
iso_no_IR_exon_region <- exon_region[exon_region$isoform_id %in% iso_no_IR,]

In [10]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+5))

acceptor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_IR_exon_region$seqnames)))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info))


In [11]:
dim(all_sites)
colnames(all_sites) <- c("site","info")
head(all_sites)


[1] 530   2

,site,info
,<chr>,<chr>
1,chr1_161954067,donor_ds_1_G
2,chr1_161954771,donor_ds_1_G
3,chr1_161968140,donor_ds_1_G
4,chr1_161970129,donor_ds_1_G
5,chr1_161976264,donor_ds_1_G
6,chr1_161987298,donor_ds_1_G


In [12]:
uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site

[1] 510   2

In [13]:
#head(all_possible_sites)
all_res <- read.table("/disk1/wenqing/tmp_data/ASD/all.res.wizSampleInfo")

head(all_res)

all_res$V10 <- paste0(all_res$V1,"_",all_res$V3)
head(all_res)

,V1,V2,V3,V4,V5,V6,V7,V8,V9
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17,chr1_99235
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17,chr1_99299
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17,chr1_99313
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17,chr1_99327
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103106
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103107


In [14]:
resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]


In [15]:
resInSpliceSites$V11 <- "info"
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info


In [17]:
head(resInSpliceSites)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
1678397,chr22,27115651,27115652,AG,4,-,CTCCTAGGTCTTCGTC,4:4,17,chr22_27115652,donor_ds_3_A_G
6090719,chr22,27115650,27115651,TC,1,+,AGGTCCGAGACGACGT,1:1,18,chr22_27115651,donor_ds_2_T
6090720,chr22,27115651,27115652,AG,2,-,CTTGGCTTCGAATGCT,1:1,18,chr22_27115652,donor_ds_3_A_G
20119120,chr22,27115651,27115652,AG,3,-,AACTCAGGTATATGGA,3:3,32,chr22_27115652,donor_ds_3_A_G


In [28]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+5))

acceptor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_no_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_no_IR_exon_region$seqnames)))

all_sites_no_IR_info <- c(donor_sites_info,acceptor_sites_info)
all_sites_no_IR <- as.data.frame(cbind(all_possible_sites,all_sites_no_IR_info))

dim(all_sites_no_IR)
colnames(all_sites_no_IR) <- c("site","info")
head(all_sites_no_IR)

uniq_all_sites_no_IR <- all_sites_no_IR[!duplicated(all_sites_no_IR$site),]
dim(uniq_all_sites_no_IR)

rownames(uniq_all_sites_no_IR) <- uniq_all_sites_no_IR$site


resInSpliceSites_no_IR <- all_res[all_res$V10 %in% all_sites_no_IR$site,]

resInSpliceSites_no_IR$V11 <- "info"
resInSpliceSites_no_IR$V11 <- uniq_all_sites_no_IR[resInSpliceSites_no_IR$V10,]$info


[1] 6300    2

,site,info
,<chr>,<chr>
1,chr1_46899620,donor_ds_1_G
2,chr1_46900921,donor_ds_1_G
3,chr1_46901709,donor_ds_1_G
4,chr1_46902126,donor_ds_1_G
5,chr1_160646295,donor_ds_1_G
6,chr1_160646295,donor_ds_1_G


[1] 5900    2

In [35]:
dim(resInSpliceSites_no_IR)

[1] 25 11

In [36]:
25/113
11/113

[1] 0.2212389

[1] 0.09734513